### ANALIZAR MÉTRICAS OBTENIDAS DE CÁLCULO DE COSTOS - BACKTEST

In [1]:
import pandas as pd
import numpy as np

from utils.utils import read_processed_data, set_root_path
from mlforecast import MLForecast
import lightgbm as lgb

from mlforecast.lag_transforms import (
    RollingMean,
    SeasonalRollingMean,
)
from sklearn.metrics import mean_absolute_error

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# set root repo
set_root_path()

root path: /Users/joseortega/Documents/GitHub/vn2_challenge


### 1. leer tablas a analizar

In [3]:
# leer output backtest estrategia optimizar costos
folder_output = "data/submission/backtest"
next_data_state_backtest = pd.read_csv(
    f"{folder_output}/next_data_state_backtest.csv"
)

In [4]:
next_data_state_backtest.head()

,Unnamed: 0,unique_id,Store,Product,Start Inventory,Sales,Missed Sales,End Inventory,In Transit W+1,In Transit W+2,Holding Cost,Shortage Cost,Cumulative Holding Cost,Cumulative Shortage Cost,fcst_w1,fcst_w2,fcst_w3,week0_update
0,0,0-126,0,126,0,0,6.0,0,0,5,0.0,6.0,0.0,6.0,4.0,4.0,3.0,2023-06-05
1,1,0-182,0,182,0,0,1.0,0,0,1,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2023-06-05
2,2,1-124,1,124,0,0,6.0,0,0,6,0.0,6.0,0.0,6.0,4.0,4.0,4.0,2023-06-05
3,3,2-124,2,124,0,0,18.0,0,0,11,0.0,18.0,0.0,18.0,9.0,10.0,8.0,2023-06-05
4,4,2-126,2,126,0,0,7.0,0,0,5,0.0,7.0,0.0,7.0,3.0,3.0,3.0,2023-06-05


### 2. Resumen costos - en la última ejecución del backtest - resumen global

In [5]:
# obtener última fecha de actualización
last_week_backtest = next_data_state_backtest["week0_update"].max()
last_week_backtest

'2024-03-18'

In [6]:
# filtrar para tener la última actualización
last_update_backtest_costs = next_data_state_backtest[next_data_state_backtest["week0_update"] == last_week_backtest]

# crear columna auxiliar total de ambos costos
last_update_backtest_costs.loc[:, "Cumulative Holding+Shortage Cost"] = last_update_backtest_costs["Cumulative Holding Cost"] + last_update_backtest_costs["Cumulative Shortage Cost"]

last_update_backtest_costs.head(3)

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_56439/1466740726.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_update_backtest_costs.loc[:, "Cumulative Holding+Shortage Cost"] = last_update_backtest_costs["Cumulative Holding Cost"] + last_update_backtest_costs["Cumulative Shortage Cost"]


,Unnamed: 0,unique_id,Store,Product,Start Inventory,Sales,Missed Sales,End Inventory,In Transit W+1,In Transit W+2,Holding Cost,Shortage Cost,Cumulative Holding Cost,Cumulative Shortage Cost,fcst_w1,fcst_w2,fcst_w3,week0_update,Cumulative Holding+Shortage Cost
24559,0,0-126,0,126,22,0,0.0,22,2,2,4.4,0.0,45.6,36.0,1.0,2.0,1.0,2024-03-18,81.6
24560,1,0-182,0,182,0,0,3.0,0,1,1,0.0,3.0,5.6,18.0,1.0,1.0,1.0,2024-03-18,23.6
24561,2,1-124,1,124,9,7,0.0,2,2,2,0.4,0.0,6.6,123.0,9.0,8.0,9.0,2024-03-18,129.6


In [7]:
# SUMAR LOS COSTOS A NIVEL "UNIQUE_ID" para tener los costos totales
cum_holding_costs = last_update_backtest_costs["Cumulative Holding Cost"].sum()
cum_shortage_costs = last_update_backtest_costs["Cumulative Shortage Cost"].sum()
cum_total_costs = last_update_backtest_costs["Cumulative Holding+Shortage Cost"].sum()

print("cum_holding_costs: ", cum_holding_costs)
print("cum_shortage_costs: ", cum_shortage_costs)
print("cum_total_costs: ", cum_total_costs)

cum_holding_costs:  12133.800000000003
cum_shortage_costs:  32123.0
cum_total_costs:  44256.8


In [ ]:
# valores usando directamente el forecast
#cum_holding_costs:  4402.6
#cum_shortage_costs:  45714.0
#cum_total_costs:  50116.6

# valores usando el forecast + holgura basada en el MAE de backtest
#cum_holding_costs:  12133.800000000003
#cum_shortage_costs:  32123.0
#cum_total_costs:  44256.8

# OJO: agregando la holgura aumentó el costo de holding pero disminuyó el de cum_shortage_costs

### 2. Resumen costos - en la última ejecución del backtest - series individualmente

In [9]:
# obtener última fecha de actualización
last_week_backtest = next_data_state_backtest["week0_update"].max()
last_week_backtest

'2024-03-18'

In [10]:
# filtrar para tener la última actualización
last_update_backtest_costs = next_data_state_backtest[next_data_state_backtest["week0_update"] == last_week_backtest]

# crear columna auxiliar total de ambos costos
last_update_backtest_costs.loc[:, "Cumulative Holding+Shortage Cost"] = last_update_backtest_costs["Cumulative Holding Cost"] + last_update_backtest_costs["Cumulative Shortage Cost"]

# ordenar por costos acumulado de mayor a menor costo
last_update_backtest_costs = last_update_backtest_costs.sort_values("Cumulative Holding+Shortage Cost", ascending=False)

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_56439/427351972.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_update_backtest_costs.loc[:, "Cumulative Holding+Shortage Cost"] = last_update_backtest_costs["Cumulative Holding Cost"] + last_update_backtest_costs["Cumulative Shortage Cost"]


In [11]:
last_update_backtest_costs

,Unnamed: 0,unique_id,Store,Product,Start Inventory,Sales,Missed Sales,End Inventory,In Transit W+1,In Transit W+2,Holding Cost,Shortage Cost,Cumulative Holding Cost,Cumulative Shortage Cost,fcst_w1,fcst_w2,fcst_w3,week0_update,Cumulative Holding+Shortage Cost
24760,201,61-23,61,23,57,57,11.0,0,50,57,0.0,11.0,463.2,1470.0,62.0,91.0,80.0,2024-03-18,1933.2
24836,277,61-124,61,124,25,25,34.0,0,80,54,0.0,34.0,47.4,1852.0,66.0,81.0,78.0,2024-03-18,1899.4
25113,554,63-124,63,124,24,24,45.0,0,106,28,0.0,45.0,99.4,1286.0,72.0,81.0,77.0,2024-03-18,1385.4
24692,133,60-125,60,125,53,53,5.0,0,56,19,0.0,5.0,90.8,1211.0,72.0,71.0,62.0,2024-03-18,1301.8
24653,94,60-23,60,23,71,30,0.0,41,38,38,8.2,0.0,497.6,718.0,46.0,35.0,50.0,2024-03-18,1215.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24923,364,61-249,61,249,2,2,0.0,0,0,0,0.0,0.0,5.0,7.0,0.0,1.0,1.0,2024-03-18,12.0
24701,142,60-166,60,166,0,0,0.0,0,0,1,0.0,0.0,6.8,5.0,0.0,1.0,1.0,2024-03-18,11.8
25088,529,62-296,62,296,1,1,0.0,0,1,0,0.0,0.0,5.8,6.0,0.0,1.0,1.0,2024-03-18,11.8
24823,264,61-101,61,101,1,1,0.0,0,0,1,0.0,0.0,3.0,7.0,0.0,1.0,1.0,2024-03-18,10.0


In [12]:
last_update_backtest_costs.head(20)

,Unnamed: 0,unique_id,Store,Product,Start Inventory,Sales,Missed Sales,End Inventory,In Transit W+1,In Transit W+2,Holding Cost,Shortage Cost,Cumulative Holding Cost,Cumulative Shortage Cost,fcst_w1,fcst_w2,fcst_w3,week0_update,Cumulative Holding+Shortage Cost
24760,201,61-23,61,23,57,57,11.0,0,50,57,0.0,11.0,463.2,1470.0,62.0,91.0,80.0,2024-03-18,1933.2
24836,277,61-124,61,124,25,25,34.0,0,80,54,0.0,34.0,47.4,1852.0,66.0,81.0,78.0,2024-03-18,1899.4
25113,554,63-124,63,124,24,24,45.0,0,106,28,0.0,45.0,99.4,1286.0,72.0,81.0,77.0,2024-03-18,1385.4
24692,133,60-125,60,125,53,53,5.0,0,56,19,0.0,5.0,90.8,1211.0,72.0,71.0,62.0,2024-03-18,1301.8
24653,94,60-23,60,23,71,30,0.0,41,38,38,8.2,0.0,497.6,718.0,46.0,35.0,50.0,2024-03-18,1215.6
25091,532,63-23,63,23,100,73,0.0,27,35,35,5.4,0.0,293.4,900.0,36.0,38.0,26.0,2024-03-18,1193.4
24977,418,62-23,62,23,48,12,0.0,36,33,33,7.2,0.0,340.0,810.0,40.0,43.0,52.0,2024-03-18,1150.0
25145,586,64-17,64,17,49,38,0.0,11,27,13,2.2,0.0,47.4,841.0,56.0,47.0,58.0,2024-03-18,888.4
25148,589,64-23,64,23,28,21,0.0,7,20,20,1.4,0.0,294.8,377.0,26.0,33.0,39.0,2024-03-18,671.8
24781,222,61-48,61,48,12,12,11.0,0,12,15,0.0,11.0,38.8,504.0,22.0,19.0,20.0,2024-03-18,542.8


### 3. Revisar todas las ejecuciones para una serie en particular

In [13]:
# param - serie a filtrar
unique_id_filter = "61-23"

In [14]:
# filtrar por la serie de interés
next_data_state_backtest[next_data_state_backtest["unique_id"] == unique_id_filter]

,Unnamed: 0,unique_id,Store,Product,Start Inventory,Sales,Missed Sales,End Inventory,In Transit W+1,In Transit W+2,Holding Cost,Shortage Cost,Cumulative Holding Cost,Cumulative Shortage Cost,fcst_w1,fcst_w2,fcst_w3,week0_update
201,201,61-23,61,23,0,0,104.0,0,0,74,0.0,104.0,0.0,104.0,10.0,28.0,24.0,2023-06-05
800,201,61-23,61,23,0,0,25.0,0,74,101,0.0,25.0,0.0,129.0,82.0,59.0,66.0,2023-06-12
1399,201,61-23,61,23,74,24,0.0,50,101,50,10.0,0.0,10.0,129.0,36.0,33.0,50.0,2023-06-19
1998,201,61-23,61,23,151,12,0.0,139,50,50,27.8,0.0,37.8,129.0,42.0,36.0,39.0,2023-06-26
2597,201,61-23,61,23,189,29,0.0,160,50,50,32.0,0.0,69.8,129.0,44.0,52.0,64.0,2023-07-03
3196,201,61-23,61,23,210,17,0.0,193,50,50,38.6,0.0,108.4,129.0,45.0,54.0,73.0,2023-07-10
3795,201,61-23,61,23,243,60,0.0,183,50,50,36.6,0.0,145.0,129.0,48.0,76.0,72.0,2023-07-17
4394,201,61-23,61,23,233,101,0.0,132,50,50,26.4,0.0,171.4,129.0,70.0,71.0,88.0,2023-07-24
4993,201,61-23,61,23,182,78,0.0,104,50,50,20.8,0.0,192.2,129.0,86.0,85.0,78.0,2023-07-31
5592,201,61-23,61,23,154,97,0.0,57,50,50,11.4,0.0,203.6,129.0,72.0,85.0,59.0,2023-08-07
